# Plant Seedlings Classification (12 Classes)

## **Import Libraries**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import cv2
import keras.backend as k
from keras.utils import np_utils


In [ ]:
!ls -lrth /kaggle/input/plant-seedlings-classification/train/
image_size = 256
batch_size = 32

image_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [ ]:
#Build training generator. 
train_generator = image_datagenerator.flow_from_directory('/kaggle/input/plant-seedlings-classification/train/',
                                                    target_size=(image_size, image_size),
                                                    subset='training',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    seed= 7
                                                )

#Build validation generator
val_generator = image_datagenerator.flow_from_directory('/kaggle/input/plant-seedlings-classification/train/',
                                                   target_size=(image_size, image_size),                                                   
                                                   subset='validation',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True,
                                                   seed= 7
                                                )

In [ ]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))

#Add 1st Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#BatchNormalization Layer
model.add(tf.keras.layers.BatchNormalization())

#Add 2nd Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

#BatchNormalization Layer
model.add(tf.keras.layers.BatchNormalization())

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (3,3)))

#Add Global Max Pool layer
model.add(tf.keras.layers.GlobalMaxPool2D())

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.2))

#Add Output Layer
model.add(tf.keras.layers.Dense(12, activation='softmax'))

In [ ]:
#Specify Loss and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#Model Summary
model.summary()

In [ ]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('plant_seedings.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history = model.fit(train_generator,
          epochs=100,
          steps_per_epoch= 3803//batch_size,  #Number of training images//batch_size
          validation_data=val_generator,
          validation_steps = 947//batch_size, #Number of validation images//batch_size
          callbacks = [model_checkpoint],
          verbose = 2
)

### Plot loss and accuracy train and validation data

In [ ]:
history_df = pd.DataFrame(history.history)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

### Evaluate the Model

In [ ]:
STEP_SIZE_VAL = val_generator.n // val_generator.batch_size
model.evaluate_generator(generator=val_generator, steps=STEP_SIZE_VAL)

### Read & Normalize test data set & Make Predictions

In [ ]:
def create_test_data(test_images_dir, image_size):
    filenames = []
    test_X = []
    for img_file in os.listdir(test_images_dir):
        filenames.append(img_file)
        img = tf.keras.preprocessing.image.load_img(test_images_dir + '/' + img_file)
        img_new = img.resize([image_size, image_size])
        img_array = tf.keras.preprocessing.image.img_to_array(img_new)
        img_array = img_array * 1./255
        test_X.append(img_array)
    test_X = np.array(test_X,dtype=np.float32)
    return test_X, filenames 

test_X, filenames  = create_test_data('/kaggle/input/plant-seedlings-classification/test/',image_size)
test_X[0].shape

In [ ]:
test_X[0]

In [ ]:
class_map = {
0:"Black-grass",
1:"Charlock",
2:"Cleavers",
3:"Common-Chickweed",
4:"Common-wheat",
5:"Fat-Hen",
6:"Loose-Silky-bent",
7:"Maize",
8:"Scentless-Mayweed",
9:"Shepherds-Purse",
10:"Small-flowered-Cranesbill",
11:"Sugar-beet",
}
prediction = []
for img in test_X:
    img_obj = np.expand_dims(img, axis=0)
    img_obj_class = model.predict(img_obj)
    img_obj_class = np.argmax(img_obj_class,axis=1)
    img_obj_class = class_map.get(img_obj_class[0], np.nan)
    prediction.append(img_obj_class)
    pass
predictions = pd.DataFrame({"image-name":filenames,"species":prediction})
predictions.to_csv('submission.csv', index = False)
predictions.head(10)